In [26]:
import pandas as pd
import xlwings as xw

In [2]:
book = xw.Book("Probalistic Loan IRR.xlsx")

In [3]:
book.sheets

Sheets([<Sheet [Probalistic Loan IRR.xlsx]Inputs and Outputs>, <Sheet [Probalistic Loan IRR.xlsx]P default>, <Sheet [Probalistic Loan IRR.xlsx]Cash Flow>])

In [4]:
sht = book.sheets[0]

In [59]:
interest_rates = sht.range("D16:D43").options(pd.DataFrame, index = False).value
interest_rates

,None
0,0.30
1,0.30
2,0.30
3,0.30
4,0.30
5,0.30
6,0.30
7,0.30
8,0.30
9,0.35


In [83]:
loan_lifes = sht.range("E16:E43").options(pd.DataFrame, index = False).value

In [84]:
initial_defaults = sht.range("F16:F43").options(pd.DataFrame, index = False).value

In [85]:
result = pd.concat([interest_rates, loan_lifes, initial_defaults], axis = 1, ignore_index=True)

In [86]:
result

,0,1,2
0,0.30,5.0,0.1
1,0.30,5.0,0.2
2,0.30,5.0,0.3
3,0.30,10.0,0.1
4,0.30,10.0,0.2
5,0.30,10.0,0.3
6,0.30,20.0,0.1
7,0.30,20.0,0.2
8,0.30,20.0,0.3
9,0.35,5.0,0.1


In [87]:
result.rename(columns={0:"Interest Rate", 1:"Loan Life", 2:"Initial Default"}, inplace = True)

In [121]:
result["Inputs"] = result.apply(lambda row: (row['Interest Rate'], row['Loan Life'], row['Initial Default']), axis=1)

In [89]:
sht_irr = book.sheets[2]

In [123]:
def irr(inputs: tuple = (.4, 20, .4)):
    interest_rate, loan_life, initial_default = inputs
    sht.range("B3").value = loan_life
    sht.range("B4").value = initial_default
    sht.range("B8").value = interest_rate
    return sht_irr.range("L13").value
    

In [125]:
def expected_value(inputs: tuple = (.4, 20, .4), iterations: int = 1000):
    lst = []
    for _ in range(iterations):
        lst.append(irr(inputs))
    return sum(lst)/len(lst)

In [128]:
def values_from_df(df):
    lst = []
    for value in df["Inputs"]:
        lst.append(expected_value(value, iterations = 100))
    return lst

In [129]:
values = values_from_df(result)

In [133]:
values_1 = values

In [142]:
def printing_results(df):
    values = values_from_df(df)
    for x in range(len(values)):
        number = 17 + x
        sht.range(f"G{number}").value = values[x]
    return None

In [144]:
printing_results(result)